<a href="https://colab.research.google.com/github/ikjot-2605/Interview_Pairing_Automate/blob/main/Interview_Pairings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

import pandas as pd

In [9]:
#interviewer class
class Interviewer:
  def __init__(self, name):
    self.name = name
    self.previouslyInterviewed = []
    self.last_interview_count = 0
    self.possible_interviewees = []
    self.assigned = 0
  def addPreviouslyInterviewed(self,interviee_name):
    self.previouslyInterviewed.append(interviee_name)
         
#interviewee class
class Interviewee:
  def __init__(self, name):
    self.name = name
    self.previouslyInterviewedBy = []
  def addPreviouslyInterviewedBy(self,interviewer_name):
    self.previouslyInterviewedBy.append(interviewer_name)      

In [10]:
sh = gc.open('Compsoc Mock Interview Pairings')
#get the worksheets
worksheet_list = sh.worksheets()
#get the length of list of worksheets
worksheet_list_length = len(worksheet_list)
#get all interviewers and interviewees 
all_interviewees = {};
all_interviewers = {};
#filling out the all_interviewer, all_interviewees dictionaries
for worksheet in worksheet_list:
  #creating dataframe for current sheet
  rows = worksheet.get_all_values()
  pd_val = pd.DataFrame.from_records(rows)
  #iterating through rows of dataframe, and building the respective dicts
  for index, row in pd_val.iterrows():
    if(index>1):
      if(row[0] not in all_interviewees):all_interviewees[row[0]] = (Interviewee(row[0]))
      if(row[1] not in all_interviewers):all_interviewers[row[1]] = (Interviewer(row[1]))
      all_interviewees[row[0]].addPreviouslyInterviewedBy(row[1])
      all_interviewers[row[1]].addPreviouslyInterviewed(row[0]) 
    
   

In [11]:
#set the previous 2 interview takers
rows = worksheet_list[len(worksheet_list)-1].get_all_values()
pd_val = pd.DataFrame.from_records(rows)
for index,row in pd_val.iterrows():
  if(index>1):
    all_interviewers[row[1]].last_interview_count=all_interviewers[row[1]].last_interview_count+1;
hash = sorted(all_interviewers.values(), key=lambda interviewer: interviewer.last_interview_count)  
newhash = {}
for interviewer in hash:
  newhash[interviewer.name] = interviewer
all_interviewers = newhash  

In [12]:
interview_pairings = []
import random
for interviewer in all_interviewers:
  for interviewee in all_interviewees:
    if not interviewee in all_interviewers[interviewer].previouslyInterviewed:
      all_interviewers[interviewer].possible_interviewees.append(all_interviewees[interviewee]) 


In [13]:
assigned_students = {}
interview_pairings = []
#assign 1 to each interviewer for now
for interviewer in all_interviewers:
  for interviewee in all_interviewers[interviewer].possible_interviewees:
    if not interviewee.name in assigned_students:
      assigned_students[interviewee.name]=1
      interview_pairings.append([interviewee.name,interviewer])
      all_interviewers[interviewer].assigned = 1
      break 
print(interview_pairings)       
for interviewee in all_interviewees:
  if interviewee in assigned_students:
    continue
  for interviewer in all_interviewers:
    lic = all_interviewers[interviewer].last_interview_count
    assigned = all_interviewers[interviewer].assigned
    print(lic)
    print(assigned)
    if (assigned>=2 or(lic==2 and assigned ==1)) or interviewer in all_interviewees[interviewee].previouslyInterviewedBy:
      continue  
    all_interviewers[interviewer].assigned = all_interviewers[interviewer].assigned+1
    interview_pairings.append([interviewee,interviewer])
    break
print(interview_pairings)
print(len(interview_pairings))    
print(len(all_interviewees))
import csv

with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(interview_pairings)

[['Sravani (IEEE)', 'Arnav Nair(IEEE)'], ['Aryaman (IEEE)', 'Vaibhav Puri(IEEE)'], ['Gaurang (IEEE)', 'Aditya Sohoni(IEEE)'], ['Harshwardhan(IEEE)', 'Krithik Vaidya(IEEE)']]
1
1
1
1
1
1
1
2
1
2
1
1
[['Sravani (IEEE)', 'Arnav Nair(IEEE)'], ['Aryaman (IEEE)', 'Vaibhav Puri(IEEE)'], ['Gaurang (IEEE)', 'Aditya Sohoni(IEEE)'], ['Harshwardhan(IEEE)', 'Krithik Vaidya(IEEE)'], ['Pranav(IEEE)', 'Vaibhav Puri(IEEE)'], ['Pratham(IEEE)', 'Arnav Nair(IEEE)'], ['Sanjkeet(IEEE)', 'Aditya Sohoni(IEEE)']]
7
7
